In [1]:
%matplotlib inline
import matplotlib
import seaborn as sns

## Summary

This notebook makes a DataFrame of weather data from 2013-01-01 to 2017-08-01, with high and low temperature, rain and snow amounts for each day.

In [2]:
import numpy as np
import scipy
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

'CHICAGO 4.7 NE, IL US' station was chosen because it was the only weather station in the Loop, but its snow data is nonexistent. For precipitation, I am using the Hyde Park weather station data, even though it is not exactly indicative of the weather in the Loop. It is as good as I can find from NAOO.

In [72]:
df_weather_raw = pd.read_csv('Data/weather_downtown.csv')

In [73]:
df_temp = df_weather_raw[df_weather_raw['NAME'] == 'CHICAGO NORTHERLY ISLAND, IL US'][['DATE', 'TMAX', 'TMIN']].set_index('DATE')

In [74]:
#df_prec = df_weather_raw[df_weather_raw['NAME'] == 'CHICAGO 4.7 NE, IL US'][['DATE', 'PRCP', 'SNOW']].set_index('DATE')

In [75]:
df_prec = pd.read_csv('Data/weather_hyde_park.csv')[['DATE', 'PRCP', 'SNOW']].set_index('DATE')

In [76]:
df_temp.head()

,TMAX,TMIN
DATE,,
2013-01-01,26.0,15.0
2013-01-02,29.0,14.0
2013-01-03,30.0,21.0
2013-01-04,37.0,16.0
2013-01-05,38.0,25.0


In [77]:
df_prec.head()

,PRCP,SNOW
DATE,,
2013-01-01,0.0,0.0
2013-01-02,0.0,0.0
2013-01-03,0.0,0.0
2013-01-04,0.0,0.0
2013-01-05,0.0,0.0


In [78]:
len(df_temp)

1668

In [79]:
len(df_prec)

1613

In [80]:
df_weather = df_temp.join(df_prec, how='outer')

These are the 2 closest stations with precipitation and temperature data, but there are some missing values. Use Midway Airport's data to complete them.

In [81]:
df_backup = pd.read_csv('Data/weather_backup.csv').set_index('DATE')

In [82]:
df_backup.head()

,STATION,NAME,AWND,PGTM,PRCP,TAVG,TMAX,TMIN,WDF2,WDF5,WSF2,WSF5,WT01,WT02,WT03,WT05,WT08,WT10
DATE,,,,,,,,,,,,,,,,,,
2013-01-01,USW00014819,"CHICAGO MIDWAY AIRPORT, IL US",8.05,1836.0,0.00,NaN,27,13,350.0,300.0,14.1,16.1,NaN,NaN,NaN,NaN,NaN,NaN
2013-01-02,USW00014819,"CHICAGO MIDWAY AIRPORT, IL US",8.95,1318.0,0.00,NaN,30,11,210.0,160.0,15.0,23.9,NaN,NaN,NaN,NaN,NaN,NaN
2013-01-03,USW00014819,"CHICAGO MIDWAY AIRPORT, IL US",11.18,1038.0,0.00,NaN,30,19,260.0,310.0,16.1,21.9,NaN,NaN,NaN,NaN,1.0,NaN
2013-01-04,USW00014819,"CHICAGO MIDWAY AIRPORT, IL US",13.87,1445.0,0.00,NaN,37,14,240.0,250.0,23.9,31.1,NaN,NaN,NaN,NaN,NaN,NaN
2013-01-05,USW00014819,"CHICAGO MIDWAY AIRPORT, IL US",10.07,1945.0,0.01,NaN,39,23,220.0,200.0,19.9,25.9,1.0,NaN,NaN,NaN,NaN,NaN


In [83]:
set(df_backup.index) - set(df_weather.index)

{'2013-05-23'}

There's one whole day missing

In [84]:
df_temp[df_temp.isnull().any(axis=1)]

,TMAX,TMIN
DATE,,
2013-05-24,NaN,NaN
2013-10-02,77.0,NaN
2013-10-03,78.0,NaN
2014-07-25,74.0,NaN
2014-09-09,NaN,NaN
2016-07-12,89.0,NaN


Snow data is often NaN. But I can replace this by 0 pretty safely (it's rare to have nonzero snow.)

In [85]:
df_prec[df_prec.isnull().any(axis=1)]

,PRCP,SNOW
DATE,,
2013-04-07,0.10,NaN
2013-04-08,0.05,NaN
2013-04-09,0.71,NaN
2013-04-10,0.05,NaN
2013-04-11,1.44,NaN
2013-04-18,3.12,NaN
2013-04-19,0.84,NaN
2013-05-05,0.05,NaN
2013-05-10,0.85,NaN


So I first fill the missing row, then

1. Use Midway data for TMAX, TMIN, PRCP

2. Use 0 for SNOW

In [86]:
df_weather.loc['2013-05-23'] = df_backup.loc['2013-05-23']

In [87]:
df_weather.loc[:, ('TMAX', 'TMIN', 'PRCP')] = df_weather.loc[:, ('TMAX', 'TMIN', 'PRCP')].fillna(df_backup)

In [88]:
df_weather.loc[:, 'SNOW'] = df_weather.loc[:, 'SNOW'].fillna(0)

There's 1 more missing value. Interpolate.

In [89]:
df_weather[df_weather.isnull().any(axis=1)]

,TMAX,TMIN,PRCP,SNOW
DATE,,,,
2014-02-26,19.0,2.0,NaN,0.0


In [90]:
df_weather = df_weather.fillna(df_weather.interpolate())

In [91]:
df_weather.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1674 entries, 2013-01-01 to 2013-05-23
Data columns (total 4 columns):
TMAX    1674 non-null float64
TMIN    1674 non-null float64
PRCP    1674 non-null float64
SNOW    1674 non-null float64
dtypes: float64(4)
memory usage: 145.4+ KB


In [92]:
df_weather.to_csv('Data/weather.csv')